## Pytorch Seq2Seq Machine Translator Practice

이번 튜토리얼에서는 Sequence to Sequence 모델의 핵심인 RNN Encoder Decoder과 Attention 모델을 이해하고, 이를 활용하여 Machine Translator를 구현해보겠습니다.

Machine Traslator에 핵심인 Sequence to Sequence 모델은 아래의 그림과 같이 구성되어 있습니다.
![alt text](img/seq2seq.png)
모델의 역할은 다음과 같습니다.
번역을 하고자 하는 데이터를 RNN Encoder에 입력하여 encoder context 정보를 얻습니다. Encoder context를 활용하여 RNN Decoder를 통해 보이고자 하는 번역 데이터를 학습하여 모델을 만듭니다. 학습된 모델은 Encoder 데이터만 입력을 하여 Decoder에서 번역된 내용을 보이게 됩니다.


### Setting Sequence Length 

Encoder와 Decoder에 입력할 최대 Sequence 길이에 대해 설정합니다. 빠른 학습을 위해서 최대길이는 10으로 지정하였습니다.

In [ ]:
MAX_LENGTH = 10

### Load Europal dataset

Europal 영-불 데이터셋을 불러옵니다.

In [ ]:
from data_util import prepare_data

input_lang, output_lang, train_pairs, test_pairs = prepare_data('lang1', 'lang2', MAX_LENGTH, 2, True)

### Implement Encoder Decoder Model

Encoder와 Decoder를 구현해봅니다. 

구현하고자 하는 Encoder는 다음과 같은 구조로 구성되어 있습니다.
![alt text](img/encoder-network.png)
위에 그래프를 보면 input vector에 대해서 embedding을 하고 hidden vector와 GRU function을 통해 한 feed forward step을 하게 됩니다. 마지막으로, GRU를 통해서 output과 hidden vector를 각각 얻게 됩니다.

다음, 구현하고자 하는 Decoder의 구조는 아래와 같습니다.
![alt text](img/attention-decoder-network.png)
Decoder구현은 Encoder보다 복잡합니다. Decoder를 보다 잘 이해하기 위해서 그래프 구조 설명 이전에 Decoder에 중요한 부분 중 하나인 Attention에 대한 설명을 하겠습니다.
![alt text](img/attn_idea.png)
Attention은 예측을 하고자 할 때 Input data에 대해 어디에 집중을 해야할 지 Encoder context에 가중치를 주는 역할을 합니다. 여기 Translator 모델에서는 매 스텝마다 들어오는 Decoder Input과 Hidden vector를 통해 Encoder context에 대한 가중치를 부여하여 Input에 대한 Output을 예측할 수 있도록 합니다.

모델의 전체과정 중 Attention 부분은 다음과 같습니다.

Input에 들어온 데이터는 embedding layer을 통해 이전 스텝의 hidden_vector와 결합을 합니다. 이후 softmax function을 거쳐 attn linear function을 두어 encoder_outputs와 matrix multiplication을 할 수 있도록 해줍니다.

Attention이 적용된 context vector는 input vector와 결합이 되어 hidden vector와 같이 GRU function에 들어갑니다. GRU에서 나온 output은 softmax를 처리하여 return 처리를 합니다.

이제 위 내용을 바탕으로 model을 구현해 보겠습니다.

models.py에 NotImplementedError라 표시된 영역에 구현해보겠습니다.

각 구현에 대한 순서는 다음과 같습니다.

1. Encoder 모델 __init__에 embedding과 gru 함수를 구현합니다.
2. Encoder 모델 forward 부분을 구현합니다. 방법은 아래와 같습니다.
    - Embedding function을 통해 word embedding layer를 구현합니다.
    - GRU function을 이용하여 multi layer RNN을 구현합니다.
3. Decoder 모델 forward 부분을 구현합니다. 방법은 아래와 같습니다.
    - Embedding functiondㅡㄹ 통해 word embedding layer를 구현합니다.
    - Attention Module을 구현합니다. (구현에 관한 내용은 위 그래프 Image를 참조하여 구현합니다.)
    - GRU function을 이용하여 multi layer RNN을 구현합니다.
    - Fully Connected Layer을 구현하고 Softmax를 통해 output data를 보입니다.

In [ ]:
from models import EncoderRNN, AttnDecoderRNN

hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,
                               MAX_LENGTH, dropout_p=0.1)

### Implement Training Module

Training Module 중 Teacher forcing 부분에 대해서 구현을 하고 criterion과 optimizer에 대해서 설정을 해봅니다.
train.py에 NotImplementedError라 표시된 영역에 구현해보겠습니다.

1. Teacher forcing 부분을 구현합니다.
    - decoder로 부터 output vector를 받습니다.
    - criterion을 활용하여 loss값을 축적합니다.
    - ground truth 값을 decoder_input에 입력합니다.
    
2. Without Teacher forcing 부분을 구현합니다.
    - decoder로 부터 ouput vector를 받습니다.
    - output vector로 부터 argmax값을 받습니다.
    - decoder로 부터 받은 예측값을 decoder_input에 입력합니다.
    - criterion을 활용하여 loss값을 축적합니다.
    - EOS_token이 있을 시 break를 하도록 조건문을 둡니다.

In [ ]:
from train import train_iters

plot_losses = train_iters(encoder1, attn_decoder1, input_lang, 
            output_lang, train_pairs[:70], 1000, MAX_LENGTH)

### Evaluate and predict model

구현한 모델의 training loss값들을 그래프로 확인하고, 번역성능을 확인해보도록 합니다.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
showPlot(plot_losses)

In [ ]:
from predict import ModelPredictor

predictor = ModelPredictor(encoder1, attn_decoder1, input_lang, output_lang, MAX_LENGTH)
predictor.evaluate_randomly(train_pairs[:10])
predictor.predict_sentence("je comprends il est essentiel .")